<a href="https://colab.research.google.com/github/shimossa/ISYS5002_Assignment_One/blob/main/Sandbox_Practice_Assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment Ideas**

1. State the problem clearly
2. Describe the input and output information
3. Work the problem by hand
4. Develop and algorithm 
5. Test slution with a variety of data

Program, Inputs, Processing and Outputs

# Inputs

*Input is a ticker symbol?

Note: To prompt the user, I guess I will need to use the input function 

**Find out what a is a ticker symbol is** 

# Processing thoughts 

*scrape the yahoo website
*extract the 'key executives'

**Find yahoo webstie**
**Find the key executives**

#Outputs

*Save to a CSV file

*Safe to an SQL database

**Workout what I have to save**
